# Data Collection Subject

How does the economy impact the popularity of movie generes?

# Scraping Data on Movies

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.the-numbers.com/market/genres'
html = requests.get(url).content
soup = BeautifulSoup(html,'lxml')

In [4]:
data = soup.find_all('table')[0].get_text()
newdata = data.split('\n')

for count in range(newdata.count('')):
    newdata.remove('')
    
newdata = newdata[1:]

genres=[]
for index in range(len(newdata)):
    if (index % 6)==1:
        item = newdata[index]
        genres.append(item)

In [5]:
newgenre=genres[:8]
for i in range(len(newgenre)):
    if newgenre[i] == 'Thriller/Suspense': 
        newgenre[i]= 'Thriller-or-Suspense'
    
    if newgenre[i] == 'Romantic Comedy':
        newgenre[i] = 'Romantic-Comedy'
    
    if newgenre[i] == 'Black Comedy':
        newgenre[i] = 'Black-Comedy'

In [6]:
#two dimensional list where dl[index] contains all of the market share datapoints by year for a genre
dl =[]
for genre in newgenre:
    #market share for each genre
    datalist=[]
    url = 'https://www.the-numbers.com/graphs/market/selected-item/genres/genre/gengres/'+genre+'/'+genre+'#_ABSTRACT_RENDERED_ID_0'
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    soup.find_all('script')[1].get_text()
    
    text =soup.find_all('script')[1].get_text()
    newtext = text.split(';')
    twotext = newtext[2].split('\t')

    for count in range(twotext.count('')):
        twotext.remove('')

    threetext = twotext[3][1:].split(', ')


    for index in range(len(threetext)):
        data = threetext[index].split(',')
        year=data[0][2:6]
        datapoint=float(data[2][:4])
        datalist.append(datapoint/100)
    
    dl.append(datalist)

#number of movies produced
dlnummade=[]
for genre in newgenre:
    url='https://www.the-numbers.com/market/genre/'+genre
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    p2text = soup.find_all('table')[1].get_text()
    makesomepoints= p2text.split('\n')

    for x in range(makesomepoints.count('')):
        makesomepoints.remove('')

    tabledata = makesomepoints[1:]
    pointsingenre=[]

    for s in range(len(tabledata)):
        if s%8==1:
            pointsingenre.append(tabledata[s])
    dlnummade.append(pointsingenre)

In [54]:
newdata=pd.DataFrame()
#append market share
index = 0
for thegenre in newgenre:
    if len(dl[index])!=26:
        dl[index].insert(0,0)
    newdata[thegenre + ' Market Share']=dl[index]
    index += 1
if len(newdata)!=24:
    newdata = newdata.iloc[:-2]

other=[]
for x in list(range(1995,2019)):
    other.append(1-final_data.loc[x][:8].sum())
newdata["Other Genres Market Share"] = other
    
gdp = pd.read_csv("gdp.csv")[(1995-1961):]
gdp.reset_index(inplace=True)
newdata['GDP % Growth']= gdp[' GDP Growth (%)']
#newdata['% GDP Annual Change']= gdp[' Annual Change']


inctitle='% Income Annual Change, Chained Dollars'
income = pd.read_csv("income.csv")
incomedata=[]
for index in range(len(income.loc[46])):
    incomedata.append(income.iloc[46][index])
newdata[inctitle] = incomedata[2:]

final_data = newdata.rename(index = lambda s: s + 1995)

In [55]:
final_data.to_csv('data.csv')

In [56]:
final_data

,Adventure Market Share,Action Market Share,Drama Market Share,Comedy Market Share,Thriller-or-Suspense Market Share,Horror Market Share,Romantic-Comedy Market Share,Musical Market Share,Other Genres Market Share,GDP % Growth,"% Income Annual Change, Chained Dollars"
1995,0.145,0.197,0.287,0.1970,0.0621,0.0278,0.0559,0.0000,0.0282,2.6843,3.3
1996,0.223,0.211,0.234,0.1800,0.0531,0.0195,0.0740,0.0004,0.0050,3.7725,3.2
1997,0.138,0.229,0.188,0.2040,0.1010,0.0641,0.0467,0.0179,0.0113,4.4472,3.7
1998,0.238,0.143,0.232,0.1160,0.1160,0.0470,0.0739,0.0099,0.0242,4.4814,5.9
1999,0.219,0.124,0.199,0.1840,0.0783,0.0650,0.0984,0.0032,0.0291,4.7532,3.3
2000,0.176,0.168,0.232,0.1960,0.0759,0.0491,0.0497,0.0123,0.0410,4.1275,5.0
2001,0.271,0.166,0.164,0.1760,0.0807,0.0468,0.0633,0.0083,0.0239,0.9983,2.7
2002,0.307,0.152,0.174,0.1420,0.1030,0.0325,0.0627,0.0063,0.0205,1.7417,3.0
2003,0.263,0.229,0.120,0.1950,0.0416,0.0490,0.0542,0.0184,0.0298,2.8612,2.7
2004,0.310,0.115,0.148,0.1800,0.0853,0.0510,0.0701,0.0024,0.0382,3.7989,3.3
